In [5]:
from pathlib import Path
import pandas as pd
from skimage import measure
import pickle
import json
from skimage import measure
import numpy as np

In [6]:
path_dict_external = (
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/notebooks/stats/maps_nnet.json"
)

with open(path_dict_external, "r") as file:
    dict_external = json.load(file)

In [3]:
path_results_internal_test = Path(
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/.nndet_crop_PRIV/inference_nndet"
)
files = list(path_results_internal_test.glob("*.pkl"))
columns = ["seriesuid", "probability", "coordZ", "coordY", "coordX", "d", "h", "w"]
rows = []
for i, files in enumerate(files):
    name_nndet = files.name
    name_orig = name_nndet.replace("case", "").replace("_boxes.pkl", "")
    name_orig = str(int(name_orig)).zfill(5)
    name_orig = f"CA_{name_orig}_0000.nii.gz"

    # read pickle
    with open(files, "rb") as f:
        data = pickle.load(f)
    boxes = data["pred_boxes"]
    scores = data["pred_scores"]
    spacing = data["itk_spacing"]

    for j, box in enumerate(boxes):
        zmin, ymin, zmax, ymax, xmin, xmax = box
        d = (zmax - zmin) * 1  # * spacing[0]
        h = (ymax - ymin) * 1  # * spacing[1]
        w = (xmax - xmin) * 1  # * spacing[2]
        coordZ = (zmax + zmin) / 2
        coordY = (ymax + ymin) / 2
        coordX = (xmax + xmin) / 2
        rows.append([name_orig, scores[j], coordZ, coordY, coordX, d, h, w])

In [4]:
path_save = (
    "~/workspace/medical/cta-det2/outputs/.nndet_crop_PRIV/inference_nndet/predict.csv"
)
df = pd.DataFrame(rows, columns=columns)
df.to_csv(path_save, index=False)

In [46]:
reverse_dict = {v: k for k, v in dict_external.items()}

In [47]:
path_results_internal_test = Path(
    "/work/vig/Datasets/aneurysm/an_nndet_preds/test_predictions_ext"
)
files = list(path_results_internal_test.glob("*.pkl"))
columns = ["seriesuid", "probability", "coordZ", "coordY", "coordX", "d", "h", "w"]
rows = []
for i, files in enumerate(files):
    name_nndet = files.name
    name_orig = reverse_dict[name_nndet.replace("_boxes.pkl", "") + ".nii.gz"]
    # read pickle
    with open(files, "rb") as f:
        data = pickle.load(f)
    boxes = data["pred_boxes"]
    scores = data["pred_scores"]
    spacing = data["itk_spacing"]

    for j, box in enumerate(boxes):
        zmin, ymin, zmax, ymax, xmin, xmax = box
        d = (zmax - zmin) * 1  # * spacing[0]
        h = (ymax - ymin) * 1  # * spacing[1]
        w = (xmax - xmin) * 1  # * spacing[2]
        coordZ = (zmax + zmin) / 2
        coordY = (ymax + ymin) / 2
        coordX = (xmax + xmin) / 2
        rows.append([name_orig, scores[j], coordZ, coordY, coordX, d, h, w])

In [48]:
path_save = (
    "~/workspace/medical/cta-det2/outputs/.nndet_crop_EXT/inference_nndet/predict.csv"
)
df = pd.DataFrame(rows, columns=columns)
df.to_csv(path_save, index=False)

In [2]:
from pathlib import Path
import pandas as pd
from skimage import measure
import pickle
import json
import numpy as np
import SimpleITK as sitk
import scipy.ndimage as ndimage
np.set_printoptions(precision=4, suppress=True)

metadata_path = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/external_crop_meta.json"
with open(metadata_path, "r") as file:
    metadata = json.load(file)

In [3]:
path_labels_test_dlca = Path("/work/vig/Datasets/aneurysm/data_dlca/test")
files = list(path_labels_test_dlca.glob("*label.nii.gz"))
len(files)

152

In [11]:
import SimpleITK as sitk

columns = ["seriesuid", "coordX", "coordY", "coordZ", "d", "h", "w", "volume"]
rows = []
for file in files:
    name = file.name.replace("_label.nii.gz", "")

    img = sitk.ReadImage(str(file))
    label_img = sitk.GetArrayFromImage(img)
    label_img, num_features = ndimage.measurements.label(
        label_img, structure=ndimage.generate_binary_structure(3, 3)
    )

    props = measure.regionprops(label_img)
    for prop in props:
        coordZ, coordY, coordX = prop.centroid
        bbox = prop.bbox
        center_z = (bbox[3] + bbox[0]) / 2
        center_y = (bbox[4] + bbox[1]) / 2
        center_x = (bbox[5] + bbox[2]) / 2
        d = bbox[3] - bbox[0]
        h = bbox[4] - bbox[1]
        w = bbox[5] - bbox[2]
        volume = prop.area
        rows.append([name, center_x, center_y, center_z, d, h, w, volume])

/tmp/ipykernel_87779/867892137.py:10: DeprecationWarning: Please import `label` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  label_img, num_features = ndimage.measurements.label(


In [8]:
len(files)

152

In [12]:
df_labels_test_dlca = pd.DataFrame(rows, columns=columns)
df_labels_test_dlca.to_csv(
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/test_dlca.csv",
    index=False,
)

In [7]:
def sigmoid_np(x):
    return 1 / (1 + np.exp(-x))


save_image = True

path_dlca_int = Path("/home/ceballosarroyo.a/workspace/medical/DLCA/test")
columns = ["seriesuid", "probability", "coordX", "coordY", "coordZ", "d", "h", "w"]
rows = []
files = list(path_dlca_int.glob("*.npy"))
for file in files[5:7]:

    name = file.name.replace("_pbb.npy", "")
    print(name)

    np_pred = np.load(file)
    for pred in np_pred:

        logit, cy, cx, cz, side = pred
        logit, cx, cy, cz, side = pred
        logit, cz, cy, cx, side = pred

        logit = sigmoid_np(logit)

        d = h = w = side * 0.5  # * spacing[0]

        rows.append([name, logit, cx, cy, cz, d, h, w])
    # new image header from orig


path_save = "/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/.dlca_crop/inference_dlca/predict.csv"
# delete file


df = pd.DataFrame(rows, columns=columns)
df.to_csv(path_save, index=False)

Ts0006.nii.gz
Ts0007.nii.gz


In [37]:
!source activate cta && cd /home/ceballosarroyo.a/workspace/medical/cta-det2/ && python src/froc.py .dlca_crop

Running iou_thr: 0.05 at dlca
parsing ground truth ...
parsing predictions ...
{'aneurysm': 126}
/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/.dlca_crop/iou0.1_froc_dlca_crop
computing froc ...
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  3.00it/s]
Per-finding bbox FROC at iou 0.05 
| Finding   | Se@FPpI=0.5   | Se@FPpI=1.0   | Se@FPpI=2.0   | Se@FPpI=4.0   | Se@FPpI=8.0   | Se@FPpI=1000.0   |
|:----------|:--------------|:--------------|:--------------|:--------------|:--------------|:-----------------|
| aneurysm  | 0.000         | 0.000         | 0.000         | 0.000         | 0.000         | 0.000            |
| Mean      | 0.000         | 0.000         | 0.000         | 0.000         | 0.000         | 0.000            |


